In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import linear_model
from sklearn.utils import shuffle

In [7]:
data = pd.read_csv("student/student-mat.csv", sep=";")

# display the first 10 rows of the data
data.head(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15
6,GP,M,16,U,LE3,T,2,2,other,other,...,4,4,4,1,1,3,0,12,12,11
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,4,1,4,1,1,1,6,6,5,6
8,GP,M,15,U,LE3,A,3,2,services,other,...,4,2,2,1,1,1,0,16,18,19
9,GP,M,15,U,GT3,T,3,4,other,other,...,5,5,1,1,1,5,0,14,15,15


### Trimming down the data to the ones we are actually going to use in this Linear Regression model


In [8]:
data = data[["G1", "G2", "G3", "studytime", "failures", "absences"]]

# attributes we are going to use here except for the final grade of which we are trying to predict so we will remove it when we are training the model

data.head(10)

# Tried to pick the ones with interger values associated with them just to make my life easier

,G1,G2,G3,studytime,failures,absences
0,5,6,6,2,0,6
1,5,5,6,2,0,4
2,7,8,10,2,3,10
3,15,14,15,3,0,2
4,6,10,10,2,0,4
5,15,15,15,2,0,10
6,12,12,11,2,0,0
7,6,5,6,2,0,6
8,16,18,19,2,0,0
9,14,15,15,2,0,0


### We are going to try to predict "G3" which is Third Grade of a student in the math class 

In [19]:
predict = "G3"  # what we are trying to predict also known as the label

#print(data["G3"])
data["G3"].head()


0     6
1     6
2    10
3    15
4    10
Name: G3, dtype: int64

### Creating the Dependent and Independent Variable

In [ ]:
x = np.array(data.drop([predict], axis=1)) # drop method normally drop rows soo the axis parameter is there to specify to drop the column rather
# This is the Independent Variable  WHich is our ATTRIBUTES

y = np.array(data[predict])
# so what we are trying to predict here is the G3 which will be on the Y-axis of Linear regression model (dependent Variable)
#WHICH IS OUR LABELS

In [ ]:
print(x)
print(y)

# they all converted to numpy arrays


[[ 5  6  2  0  6]
 [ 5  5  2  0  4]
 [ 7  8  2  3 10]
 ...
 [10  8  1  3  3]
 [11 12  1  0  0]
 [ 8  9  1  0  5]]
[ 6  6 10 15 10 15 11  6 19 15  9 12 14 11 16 14 14 10  5 10 15 15 16 12
  8  8 11 15 11 11 12 17 16 12 15  6 18 15 11 13 11 12 18 11  9  6 11 20
 14  7 13 13 10 11 13 10 15 15  9 16 11 11  9  9 10 15 12  6  8 16 15 10
  5 14 11 10 10 11 10  5 12 11  6 15 10  8  6 14 10  7  8 18  6 10 14 10
 15 10 14  8  5 17 14  6 18 11  8 18 13 16 19 10 13 19  9 16 14 13  8 13
 15 15 13 13  8 12 11  9  0 18  0  0 12 11  0  0  0  0 12 15  0  9 11 13
  0 11  0 11  0 10  0 14 10  0 12  8 13 10 15 12  0  7  0 10  7 12 10 16
  0 14  0 16 10  0  9  9 11  6  9 11  8 12 17  8 12 11 11 15  9 10 13  9
  8 10 14 15 16 10 18 10 16 10 10  6 11  9  7 13 10  7  8 13 14  8 10 15
  4  8  8 10  6  0 17 13 14  7 15 12  9 12 14 11  9 13  6 10 13 12 11  0
 12 12  0 12  0 18 13  8  5 15  8 10  8  8 12  8 13 11 14  0 18  8 12  9
  0 17 10 11 10  0  9 14 11 14 10 12  9  9  8 10  8 10 12 10 11 11 19 12
 14 15 11 

### Converting our X & Y variables into 4 parts (X_Train,Y_Train) & (X_Test, Y_Test)

In [31]:
import sklearn.model_selection


# Splitting them up into 4 different arrays

x_train,y_train,x_test,y_test = sklearn.model_selection.train_test_split(x,y,test_size=0.1)